In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time
import math
import yaml
from collections import OrderedDict
import os
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Sensitivity Analysis/SensitivityAnalysisResnet56/Element/sensitivity.csv")
print(df.head)

<bound method NDFrame.head of                 parameter  sparsity   top1   top5      loss
0     module.conv1.weight      0.00  92.94  99.84  0.338810
1     module.conv1.weight      0.05  92.94  99.84  0.354796
2     module.conv1.weight      0.10  92.94  99.84  0.340292
3     module.conv1.weight      0.15  92.94  99.84  0.338813
4     module.conv1.weight      0.20  92.94  99.84  0.339793
...                   ...       ...    ...    ...       ...
1097     module.fc.weight      0.70  91.99  99.65  0.369765
1098     module.fc.weight      0.75  91.70  99.63  0.352319
1099     module.fc.weight      0.80  91.51  99.57  0.332645
1100     module.fc.weight      0.85  90.56  99.30  0.361597
1101     module.fc.weight      0.90  90.05  99.24  0.345115

[1102 rows x 5 columns]>


In [ ]:
#get unique layers from model
layers = df.parameter.unique().tolist()
layerResults = {}
curIndex = 0
curResult = []
for index,row in df.iterrows():
  if (row[0] != layers[curIndex]):
    layerResults[layers[curIndex]] = curResult
    curResult = []
    curIndex += 1
  curResult.append((row[1],row[2]))
layerResults[layers[curIndex]] = curResult

In [ ]:
baseAcc = 92.94
def findSparsities(threshold, scaling = 1):
  perLayerSparsity = []
  for layer, results in layerResults.items():
    bestSparsity = 0
    for sparsity, accuracy in results:
      if accuracy + threshold >= baseAcc:
        bestSparsity = sparsity
    perLayerSparsity.append((layer,bestSparsity * scaling))
  return perLayerSparsity
schedule1 = findSparsities(0.00)
schedule2 =  findSparsities(0.5)
schedule3 = findSparsities(1)
schedule4 = findSparsities(0.1)
schedule5 = findSparsities(0.00, 0.9)
schedule6 = findSparsities(0.00, 0.8)

In [ ]:
class LevelPruner:
  def __init__(self, perLayerSparsity):
    self.pruner1 = {}
    self.pruner1["class"] = "SparsityLevelParameterPruner"
    self.pruner1["levels"] = []
    for name, sparsity in perLayerSparsity:
      self.pruner1["levels"].append({name: round(sparsity, 4)})

class LevelSchedule:
  def __init__(self, perLayerSparsity):
    self.version = 1
    self.pruners = LevelPruner(perLayerSparsity)

schedule1YAML = LevelSchedule(schedule1)
f = open("threshold0.yaml", "w")
yaml.dump(schedule1YAML, f, default_flow_style=False)

schedule2YAML = LevelSchedule(schedule2)
f = open("threshold0-5.yaml", "w")
yaml.dump(schedule2YAML, f, default_flow_style=False)

schedule3YAML = LevelSchedule(schedule3)
f = open("threshold1.yaml", "w")
yaml.dump(schedule3YAML, f, default_flow_style=False)

schedule4YAML = LevelSchedule(schedule4)
f = open("threshold0-1.yaml", "w")
yaml.dump(schedule4YAML, f, default_flow_style=False)

# schedule5YAML = LevelSchedule(schedule5)
# f = open("threshold0Scale0-9.yaml", "w")
# yaml.dump(schedule5YAML, f, default_flow_style=False)

# schedule6YAML = LevelSchedule(schedule6)
# f = open("threshold0Scale0-8.yaml", "w")
# yaml.dump(schedule6YAML, f, default_flow_style=False)

In [ ]:
def addQuotes(filename):
  with open(filename) as f:
    content = f.readlines()
  newName = filename[:-5] + "NoLR.yaml"
  outF = open(newName, "w")
  addQuotes = False
  finalContent = []
  newLine = None
  for line in content:
    newLine = line
    if "levels" in line:
      addQuotes = True
    elif addQuotes:
      index0 = line.find("-")
      index1 = line.find("m")
      index2 = line.find(":")
      if index1 != -1:
        newLine = line[:index0] + " " + line[index0 + 1:index1] + "'" + line[index1:index2] + "'" + line[index2:]
    finalContent.append(newLine)
  for line in finalContent:
    outF.write(line)
addQuotes("threshold0.yaml")
addQuotes("threshold0-5.yaml")
addQuotes("threshold1.yaml")
addQuotes("threshold0-1.yaml")